<a href="https://colab.research.google.com/github/juanjogomez/MUNE_NeuroProsthetics/blob/main/MUNE_VisualProsthesis_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎥 **Visual Prosthesis Simulation Notebook**

## Simulating Phosphene-Based Vision in Visual Prostheses

This notebook guides you step by step to understand and replicate the experience of a visual prosthesis using phosphene dots. It enables users to simulate how different neuroprosthetic devices process visual input and translate it into artificial sight.

### 🔬 Master in Neurotechnology - Universidad Politécnica de Madrid
#### 🏥 Neuroprosthetics Course
#### 📅 March 2025


# Step 1: Install Dependencies

In [1]:
!pip install opencv-python numpy moviepy

# Step 2: Import Required Libraries

In [2]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import moviepy.editor as mp
import tempfile
import os
from google.colab import files
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import datetime

  if event.key is 'enter':



# Step 3: Create directories for input and ouput


In [3]:
base_dir = "/content/MUNE_NeuroProthestics_PhosphenesSimulator"
input_dir = os.path.join(base_dir, "input")
output_dir = os.path.join(base_dir, "output")

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# Step 4: Define Visual Prosthesis Settings


In [4]:
prosthesis_presets = {
    "Dobelle Eye": {"num_phosphenes": 40, "visual_field": 30, "noise_level": 0.3},
    "Argus II": {"num_phosphenes": 50, "visual_field": 20, "noise_level": 0.3},
}

prosthesis_selector = widgets.Dropdown(
    options=list(prosthesis_presets.keys()),
    value="Argus II",
    description="Prosthesis:",
)

# Step 5: Define Adjustable Parameters

This section allows you to adjust key parameters for the visual prosthesis simulation. You can either select a preset configuration or manually modify the sliders.

🔍 **How It Works: update_parameters**

The preset selector (prosthesis_selector) lets you choose a predefined setting. When you select a preset, the values for:
*   Phosphene Count (number of visual stimuli)
*   Visual Field (in degrees)
*   Noise Level (amount of added distortion)
will automatically update.

If you prefer, you can manually adjust these values using the sliders.

🛠 **Adjustable Parameters**

| **Parameter**          | **Description**                                      | **Widget Type**  |
|------------------------|------------------------------------------------------|------------------|
| `num_phosphenes`      | Number of phosphenes (light spots)                   | IntSlider (50-2000) |
| `visual_field`        | Field of view in degrees                             | IntSlider (10°-100°) |
| `noise_level`         | Level of noise affecting the image                   | FloatSlider (0-1) |
| `edge_detection`      | Enhances edges of objects for better visibility      | Checkbox |
| `circular_mask_option` | Applies a circular mask to limit the field of view | Checkbox |

🔍 **How It Works:  Phosphene Grid & Circular Mask Generation**

This section explains how the two methods **`create_phosphene_grid`** and **`create_circular_mask`** generate different visual patterns in the prosthesis simulation.

---

##### 🟢 1️⃣ `create_phosphene_grid(h, w, num_phosphenes=500, dot_radius=5)`
This function **generates a grid of phosphenes** (small bright spots) distributed across the image in a jittered pattern to simulate prosthetic vision.
1. **Creates a blank mask** (`mask`) with dimensions `(h, w)`, where `0` represents background and `255` represents phosphenes.
2. **Calculates grid spacing**:
   - The image area `(h * w)` is divided by `num_phosphenes` to determine how far apart phosphenes should be.
   - `grid_spacing` is derived as the square root of this division.
3. **Places phosphenes with slight randomness**:
   - Iterates over the image in steps of `grid_spacing` (both `x` and `y`).
   - Introduces **random jitter** within ±25% of the grid spacing to avoid a strict pattern.
   - Ensures positions stay within valid image bounds using `np.clip`.
   - Draws a small **white circle** at each phosphene position using `cv2.circle()`.
4. **Returns a boolean mask** (`True` for phosphenes, `False` for the background).

##### 📌 Example Output:
This function creates a **scattered grid** of phosphenes across the screen.
- Increasing `num_phosphenes` makes the grid denser.
- A larger `dot_radius` creates bigger phosphenes.

---

#####  🔵 2️⃣ `create_circular_mask(h, w, center=None, radius=None)`
This function generates a **circular mask** to limit the visual field to a circular area.

1. **Determines center and radius**:
   - If no `center` is provided, the default is the **middle of the image** (`(w/2, h/2)`).
   - If no `radius` is provided, it is set to the largest possible value that keeps the circle **inside the image**.
2. **Creates coordinate grids**:
   - `np.ogrid[:h, :w]` generates arrays for **X and Y coordinates** of each pixel.
   - The **distance of each pixel** from the center is calculated using the Euclidean formula:  
     $$
     \text{distance} = \sqrt{(X - x_{\text{center}})^2 + (Y - y_{\text{center}})^2}
     $$
3. **Generates the circular mask**:
   - Pixels **inside the radius** are set to `True`, while those outside are `False`.
4. **Returns the boolean mask**, where `True` represents the area **inside the circle**.

##### 📌 Example Output:
- This function creates a **circular aperture** effect, blocking out the peripheral areas.
- The result is a **restricted field of view**, similar to tunnel vision.

---

#####  🔄 How These Functions Work Together:
- **`create_phosphene_grid`** generates the **phosphene layout**.
- **`create_circular_mask`** can be applied **on top of it** to restrict vision to a circular region.



In [15]:
def update_parameters(change):
    selected_preset = prosthesis_presets[prosthesis_selector.value]
    num_phosphenes.value = selected_preset["num_phosphenes"]
    visual_field.value = selected_preset["visual_field"]
    noise_level.value = selected_preset["noise_level"]

prosthesis_selector.observe(update_parameters, names='value')

num_phosphenes = widgets.IntSlider(min=20, max=5000, step=50, value=60, description='Phosphene Count')
visual_field = widgets.IntSlider(min=10, max=100, step=10, value=20, description='Visual Field (°)')
noise_level = widgets.FloatSlider(min=0, max=1, step=0.05, value=0.3, description='Noise Level')
edge_detection = widgets.Checkbox(value=False, description="Edge Detection")
circular_mask_option = widgets.Checkbox(value=False, description="Use Circular Mask")

In [6]:
def create_phosphene_grid(h, w, num_phosphenes=500, dot_radius=5):
    mask = np.zeros((h, w), dtype=np.uint8)
    grid_spacing = int(np.sqrt((h * w) / num_phosphenes))

    for y in range(0, h, grid_spacing):
        for x in range(0, w, grid_spacing):
            jitter_x = np.random.randint(-grid_spacing//4, grid_spacing//4)
            jitter_y = np.random.randint(-grid_spacing//4, grid_spacing//4)
            x_pos = np.clip(x + jitter_x, 0, w-1)
            y_pos = np.clip(y + jitter_y, 0, h-1)
            cv2.circle(mask, (x_pos, y_pos), radius=dot_radius, color=255, thickness=-1)

    return mask > 0

def create_circular_mask(h, w, center=None, radius=None):
    if center is None:
        center = (int(w / 2), int(h / 2))
    if radius is None:
        radius = min(center[0], center[1], w - center[0], h - center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    mask = dist_from_center <= radius
    return mask

# Step 6: Define the Phosphene Simulation Function
🔍 **How It Works: Phosphene-Based Vision Simulation**

This function, **`phosphene_simulation`**, takes a video frame as input and processes it to simulate vision through a visual prosthesis. It applies different effects such as **phosphenes, edge detection, noise, and a circular mask** to replicate the experience of prosthetic vision.

---

##### 🟢 `phosphene_simulation(frame, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask)`

This function **transforms an image into a phosphene-based representation** by applying different processing techniques.

1️⃣ **Preprocessing the Input Image**  
   - The function extracts the height `h` and width `w` of the input **frame**.  
   - Converts the **color frame** to **grayscale** using:
     ```python
     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     ```

2️⃣ **Optional Edge Detection (Enhances Object Boundaries)**  
   - If `edge_detection` is **enabled**, the function applies the **Canny edge detection** filter:
     ```python
     gray_frame = cv2.Canny(gray_frame, 50, 150)
     ```
   - This highlights important contours in the image.

3️⃣ **Generating the Phosphene Grid (Sparse Vision Representation)**  
   - The function calls **`create_phosphene_grid()`** to generate a mask defining **phosphene positions**:
     ```python
     phosphene_mask = create_phosphene_grid(h, w, num_phosphenes)
     ```

4️⃣ **Applying a Flickering Effect to Simulate Phosphene Variability**  
   - A **random flickering effect** is applied to phosphenes, mimicking real-life variations in perception:
     ```python
     flicker_effect = np.random.uniform(0.7, 1.2, size=(h, w))
     ```
   - The **grayscale intensity** is modified using this effect:
     ```python
     phosphene_image[phosphene_mask] = (gray_frame[phosphene_mask] * flicker_effect[phosphene_mask]).astype(np.uint8)
     ```

5️⃣ **Applying the Circular Mask (Restricted Field of View)**  
   - If `circular_mask` is **enabled**, the function creates a **circular viewing area**:
     ```python
     mask = create_circular_mask(h, w, radius=int((visual_field / 180) * (min(h, w) / 2)))
     ```
   - The final image is **masked** so only the area within the circular field of view remains visible.

6️⃣ **Returning the Processed Image**  
   - Converts the processed grayscale image back to **BGR format**:
     ```python
     return cv2.cvtColor(final_image, cv2.COLOR_GRAY2BGR)
     ```
---

#### 📌 Example Output:
- If **edge detection** is enabled, the image will have **sharp contours** of objects.
- If **circular masking** is enabled, only a **central portion** of the image will be visible.
- **Phosphenes appear as randomly positioned light spots**, simulating prosthetic vision.

---

#### 🔄 How This Function Works with Other Components:
- **`create_phosphene_grid()`**: Defines where phosphenes appear.
- **`create_circular_mask()`**: Restricts vision to a circular region.
- **Flicker effect**: Simulates real-life phosphene instability.



In [7]:
def phosphene_simulation(frame, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask):
    h, w, _ = frame.shape
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if edge_detection:
        gray_frame = cv2.Canny(gray_frame, 50, 150)

    phosphene_mask = create_phosphene_grid(h, w, num_phosphenes)
    flicker_effect = np.random.uniform(0.7, 1.2, size=(h, w))
    phosphene_image = np.zeros_like(gray_frame)
    phosphene_image[phosphene_mask] = (gray_frame[phosphene_mask] * flicker_effect[phosphene_mask]).astype(np.uint8)

    if circular_mask:
        mask = create_circular_mask(h, w, radius=int((visual_field / 180) * (min(h, w) / 2)))
        final_image = np.zeros_like(phosphene_image)
        final_image[mask] = phosphene_image[mask]
    else:
        final_image = phosphene_image

    return cv2.cvtColor(final_image, cv2.COLOR_GRAY2BGR)


# Step 7: Process the Video
🔍 **How It Works: Processing a Video for Phosphene-Based Vision**

The **`process_video`** function takes a video file, processes each frame to simulate vision through a visual prosthesis, and saves the output as a new video. It applies phosphenes, noise, edge detection, and a circular mask to replicate the prosthetic vision experience.

---

##### 🟢 `process_video(video_path, output_path, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask)`

1️⃣ **Load the Input Video**
   - Opens the video using OpenCV:
     ```python
     cap = cv2.VideoCapture(video_path)
     ```
   - Retrieves important **video properties** like:
     - `fps` (frames per second)
     - `width` and `height` (frame dimensions)

2️⃣ **Prepare the Output Video Writer**
   - Initializes the **MP4 video writer** to store the processed frames:
     ```python
     out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
     ```
   - The output video has the same **frame rate and resolution** as the original.

3️⃣ **Process Each Frame of the Video**
   - Reads frames one by one in a loop:
     ```python
     while cap.isOpened():
         ret, frame = cap.read()
         if not ret:
             break
     ```
   - Calls the **`phosphene_simulation()`** function to transform each frame:
     ```python
     processed_frame = phosphene_simulation(frame, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask)
     ```
   - Writes the **processed frame** into the output video file:
     ```python
     out.write(processed_frame)
     ```

4️⃣ **Finalize and Save the Video**
   - Closes the video file and releases memory:
     ```python
     cap.release()
     out.release()
     ```
   - Prints the **path of the saved video** and the parameters used.

---

#### 📌 Example Output:
- The saved video will display **phosphene-based vision**, where:
  - **Edge detection (if enabled)** highlights object boundaries.
  - **Circular mask (if enabled)** restricts the field of view.
  - **Phosphenes appear in a scattered pattern**, simulating artificial vision.

---

#### 🔄 How This Function Works with Other Components:
- **`phosphene_simulation()`**: Processes each frame.
- **`create_phosphene_grid()`**: Defines where phosphenes appear.
- **`create_circular_mask()`**: Applies a field-of-view limitation.



In [8]:
def process_video(video_path, output_path, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        processed_frame = phosphene_simulation(frame, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask)
        out.write(processed_frame)

    cap.release()
    out.release()

    print(f"Processed video saved at: {output_path}")
    print(f"Parameters Used: Phosphene Count = {num_phosphenes}, Noise Level = {noise_level}, Edge Detection = {edge_detection}, Circular Mask = {circular_mask}")


# Step 8: Display the Processed Video
🔍 **How It Works: Generating a Phosphene-Based Video**

The **`generate_phosphenes`** function automates the process of applying the phosphene simulation to a video. It creates a **timestamped output filename**, retrieves the selected prosthesis parameters, and processes the video accordingly.

---

##### 🟢 `generate_phosphenes(_)`

1️⃣ **Generate a Unique Filename**  
   - Creates a **timestamp** (date and time) to ensure that each output file has a unique name:
     ```python
     timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
     ```
   - Retrieves the **selected prosthesis type** and formats it for the filename:
     ```python
     prosthesis_name = prosthesis_selector.value.replace(" ", "_")
     ```
   - Combines everything into a structured filename:
     ```python
     output_filename = f"{prosthesis_name}_phosphene_output_{timestamp}.mp4"
     ```

2️⃣ **Define the Output Path**  
   - Joins the generated filename with the output directory:
     ```python
     output_path = os.path.join(output_dir, output_filename)
     ```

3️⃣ **Run the Video Processing Function**  
   - Calls `process_video()` to apply phosphene simulation to the input video:
     ```python
     process_video(input_video_path, output_path, num_phosphenes.value, noise_level.value, visual_field.value, edge_detection.value, circular_mask_option.value)
     ```
   - Uses **values from interactive widgets** to set parameters dynamically.

---

#### 📌 Example Output
- Saves a **processed video** in the output directory with a **timestamped filename**.
- The video will display **phosphene-based artificial vision**, with adjustments based on user-selected values.
- Example filename: ArgusII_phosphene_output_20250309_153045.mp4

---

#### 🔄 How This Function Works with Other Components:
- **`process_video()`**: Handles the actual frame-by-frame phosphene transformation.
- **`phosphene_simulation()`**: Applies phosphene and masking effects.
- **Widget values (`num_phosphenes`, `noise_level`, etc.)**: Allow real-time parameter customization.




In [9]:
def generate_phosphenes(_):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    prosthesis_name = prosthesis_selector.value.replace(" ", "_")
    output_filename = f"{prosthesis_name}_phosphene_output_{timestamp}.mp4"
    output_path = os.path.join(output_dir, output_filename)
    process_video(input_video_path, output_path, num_phosphenes.value, noise_level.value, visual_field.value, edge_detection.value, circular_mask_option.value)


In [10]:
# Ensure inline HTML5 animations work in Matplotlib
plt.rcParams["animation.html"] = "html5"

def display_video(video, fps=20):
    """
    Displays an inline video animation in Google Colab using Matplotlib.

    Parameters:
        video (list): List of video frames.
        fps (int): Frames per second for animation speed.

    Returns:
        HTML5 video string.
    """
    fig, ax = plt.subplots(figsize=(5,5))
    ax.axis('off')  # Hide axis labels

    frames = [[ax.imshow(frame, animated=True)] for frame in video]

    # Create and return the animation
    ani = animation.ArtistAnimation(fig, frames, interval=1000 / fps, repeat_delay=1000, blit=True)
    plt.close(fig)  # Prevent duplicate figure output
    return ani.to_html5_video()


def load_video(video_path, target_size=256):
    try:
        # Read video metadata
        reader = imageio.get_reader(video_path, 'ffmpeg')
        fps = reader.get_meta_data().get('fps', 20)
        original_width, original_height = reader.get_meta_data().get('size', (None, None))

        # Handle cases where size metadata is missing
        if not original_width or not original_height:
            raise ValueError("Video size metadata is missing.")

        # Maintain aspect ratio
        if original_width > original_height:
            new_width = target_size
            new_height = int(original_height * (target_size / original_width))
        else:
            new_height = target_size
            new_width = int(original_width * (target_size / original_height))

        # Load and resize frames
        video = [resize(frame, (new_height, new_width))[..., :3] for frame in reader]
        return video, fps

    except Exception as e:
        print(f"Error loading video: {e}")
        return None, None


# Step 9: Run the Application

In [11]:
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]
input_path = os.path.join(input_dir, video_filename)
os.rename(video_filename, input_path)

# Store the input path globally
global input_video_path
input_video_path = input_path

Saving video_people_walking.mp4 to video_people_walking.mp4


In [16]:
button = widgets.Button(description="Generate Phosphene Video")
button.on_click(generate_phosphenes)

display(prosthesis_selector, num_phosphenes, noise_level, visual_field, edge_detection, circular_mask_option, button)


Dropdown(description='Prosthesis:', index=1, options=('Dobelle Eye', 'Argus II'), value='Argus II')

IntSlider(value=60, description='Phosphene Count', max=5000, min=20, step=50)

FloatSlider(value=0.3, description='Noise Level', max=1.0, step=0.05)

IntSlider(value=20, description='Visual Field (°)', min=10, step=10)

Checkbox(value=False, description='Edge Detection')

Checkbox(value=False, description='Use Circular Mask')

Button(description='Generate Phosphene Video', style=ButtonStyle())

In [14]:
# Load the video
video_path = "/content/MUNE_NeuroProthestics_PhosphenesSimulator/output/Argus_II_phosphene_output_20250310_094411.mp4"  # Change this if needed
video, fps = load_video(video_path)

# Display video if successfully loaded
if video:
    display(HTML(display_video(video, fps=fps)))
else:
    print("Error: Could not load video.")


# Step 10 Cleanup the folders/files

In [21]:
def cleanup_folders(_):
    import shutil
    remove_folders = remove_folders_checkbox.value

    shutil.rmtree(input_dir, ignore_errors=True)
    shutil.rmtree(output_dir, ignore_errors=True)

    if remove_folders:
        shutil.rmtree(base_dir, ignore_errors=True)
        print("All files and folders have been deleted.")
    else:
        os.makedirs(input_dir, exist_ok=True)
        os.makedirs(output_dir, exist_ok=True)
        print("All input and output files have been deleted, but folders remain.")

remove_folders_checkbox = widgets.Checkbox(value=False, description="Remove All Folders")
cleanup_button = widgets.Button(description="Clean Up Files")
cleanup_button.on_click(cleanup_folders)

display(remove_folders_checkbox, cleanup_button)

Checkbox(value=False, description='Remove All Folders')

Button(description='Clean Up Files', style=ButtonStyle())

All files and folders have been deleted.
